In [2]:
!pip install transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/07/78/c23e1c70b89f361d855a5d0a19b229297f6456961f9a1afa9a69cd5a70c3/transformers-4.41.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ---------------------------------------- 43.8/43.8 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers<0.20,>=0.19 from https://files.pythonhosted.org/packages/f4/85/d999b9a05fd101d48f1a365d68be0b109277bb25c89fb37a389d669f9185/tokenizers-0.19.1-cp310-none-win_amd64.whl.metadata
  Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
    --------------------------------------- 0.1/9.1 MB 4.3 MB/s eta 0:00:03
   - -------------------------------------- 0.4/9.1 MB 4.6 MB/s eta 0:00:02
   --- ------------------------------------ 0.7/9.1 MB 5.6 MB/s eta 0:00:02
   ---- -------------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

model_id = "stabilityai/stable-diffusion-2"

scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
# pipe = pipe.to("cuda")

ImportError: 
StableDiffusionPipeline requires the transformers library but it was not found in your environment. You can install it with pip: `pip
install transformers`


In [ ]:
prompt = "oil painting of the Leaning Tower of Pisa at night with the moon"
image = pipe(prompt).images[0]
image.save("night.png")

In [ ]:
prompt = "oil painting of the Leaning Tower of Pisa during the sunset"
image = pipe(prompt).images[0]
image.save("sunset.png")

In [ ]:
from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
from tqdm import trange

with torch.no_grad():
    start_image = Image.open("night.png")
    start_image = TF.resize(start_image, (256, 256))
    start_image = TF.to_tensor(start_image)
    start_image = start_image.unsqueeze(0).half().cuda()

    end_image = Image.open("sunset.png")
    end_image = TF.resize(end_image, (256, 256))
    end_image = TF.to_tensor(end_image)
    end_image = end_image.unsqueeze(0).half().cuda()


    start_latent_space = pipe.vae.encode(start_image)
    start_sample = start_latent_space.latent_dist.mean

    end_latent_space = pipe.vae.encode(end_image)
    end_sample = end_latent_space.latent_dist.mean

    direction = end_sample - start_sample

    frames = []
    steps = 150
    for i in trange(0, steps+1):
        sample = start_sample + direction * (i / steps)
        out = pipe.vae.decode(sample)
        out = out.sample[0].float().cpu().detach().numpy().transpose(1, 2, 0)
        out = (out * 255).astype("uint8")
        frames.append(out)


In [ ]:
import imageio
imageio.mimsave('morph.gif', frames, fps=30)